In [ ]:
# setup en google colab
# descoméntalo si estás en dicho entorno
# !git clone https://github.com/koldLight/python-data-ull
# %cd python-data-ull

# Pivotación de tablas

Vamos a ver cómo transformar las tablas de formato ancho a largo y viceversa

In [ ]:
import pandas as pd

In [ ]:
air = pd.read_csv("dat/airquality.csv")
air.head()

In [ ]:
len(air)

## Melt: de ancho a largo

Para pasar de formato ancho a largo, podemos usar [`melt`](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.melt.html)

In [ ]:
air_long = air.melt(id_vars=['month', 'day'])
air_long

In [ ]:
len(air_long)

Vemos que, para cada mes y día, ahora contamos con dos columnas: la variable medida y su valor.

En el formato largo, cada fila cuenta con el índice (en este caso, mes y día), un valor, y etiquetas del valor.

## Pivot: de largo a ancho

Para pasar de formato largo a ancho, podemos usar [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [ ]:
air_wide = air_long.pivot_table(index=['month', 'day'], columns='variable', values='value')
air_wide.head()

Los índices jerárquicos suelen ser incómodos para tratar la tabla. Podemos quitarlo con `reset_index()`

In [ ]:
air_wide = air_wide.reset_index()
air_wide.head()

In [ ]:
# Si además queremos quitar el nombre "feo" que se ha quedado como nombre de las columnas...
air_wide.columns.name = ''
air_wide.head()

#### Ejercicio

Investiga y cambia los nombres de las columnas resultantes (variable y value) por otros que consideres adecuados, durante el proceso de `melt`.

#### Ejercicio

La función [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) permite agregar si hay más de un valor por cada fila (es decir, parejas de índice + variable duplicadas).

Aprovecha esta utilidad para, en el paso de `air_long` a ancho, calcular la mediana por variable por mes.

#### Ejemplo del mundo real

Examina los datos contenidos en el fichero `dat/contaminacion_mad_201812.csv`, y mira qué aspecto tiene. Nuestro objetivo es dejarlo en formato largo, y quedarnos únicamente con los días con una medida válida.

Para hacerlo, podemos dividir el problema en los siguientes pasos:

* A partir de ese dataframe, crear dos nuevos:

    * Uno con el índice + las columnas con los valores (D01, ..., D31)
    * Otro con el índice + las columnas con la validez (V01, ..., V31)

* Pasar ambos a formato largo. Es decir, queremos tener una columna `dia` y otra `validez` y que los valores pasen a ser filas, en lugar de tener 31 columnas. El día debe quedar como numérico en ambas tablas.
* Cruzar los resultados, para tener en el mismo dataframe valor y validez.
* Limpiar la fecha, para tenerla como `date`.
* Castear la validez a booleano (True / False).

#### Referencia

Puedes consultar la guía completa de pandas sobre pivotación [aquí](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)